<a href="https://colab.research.google.com/github/sophiawang705/AISF-Bipedal-Walker/blob/main/AISF_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gymnasium[box2d] stable_baselines3 shimmy

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for box2d-py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
Failed to build box2d-py
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> box2d-py


In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import os

# 1. Setup Environment
env_id = "BipedalWalker-v3"
# For 'hardcore', change to "BipedalWalkerHardcore-v3"
env = gym.make(env_id, render_mode="rgb_array")
env = DummyVecEnv([lambda: env])

# CRITICAL: Normalize observations and rewards for stability
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 2. Define the Model (MLP Policy)
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01, # Helps with exploration
    tensorboard_log="./ppo_walker_tensorboard/"
)

# 3. Train (Start with 100k steps for baseline, target 1M+ for solution)
print("Training...")
model.learn(total_timesteps=100000)

# 4. Save the model and statistics
model.save("ppo_bipedal_walker")
env.save("vec_normalize.pkl")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:315

Using cpu device
Training...
Logging to ./ppo_walker_tensorboard/PPO_1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------
| time/              |      |
|    fps             | 611  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 567         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011883976 |
|    clip_fraction        | 0.0827      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.72       |
|    explained_variance   | -0.102      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.196       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    std                  | 1.02        |
|    value_loss           | 1.29        |
----------------------------------

In [3]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model_path, env_id, video_length=500, prefix="walker-video"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Load normalization stats from your training
    eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
    # Stop updating normalization stats during evaluation
    eval_env.training = False
    eval_env.norm_reward = False

    eval_env = VecVideoRecorder(eval_env, "./videos/",
                                record_video_trigger=lambda x: x == 0,
                                video_length=video_length,
                                name_prefix=prefix)

    model = PPO.load(model_path, env=eval_env)
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = eval_env.step(action)

    eval_env.close()

# Run the recording
record_video("ppo_bipedal_walker", "BipedalWalker-v3")

Saving video to /content/videos/walker-video-step-0-to-step-500.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Building video /content/videos/walker-video-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/walker-video-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/walker-video-step-0-to-step-500.mp4


In [5]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width=400 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """)

show_video('/content/videos/walker-video-step-0-to-step-500.mp4')

In [6]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.evaluation import evaluate_policy

env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)

# --- TRAIN ENV (NO RENDER) ---
train_env = DummyVecEnv([lambda: gym.make(env_id)])
train_env = VecMonitor(train_env)

# Normalize obs + rewards for training stability
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)

model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("Training...")
model.learn(total_timesteps=100_000_000)

model.save("models/ppo_bipedalwalker_vecnorm")
train_env.save("models/vecnormalize.pkl")

# --- EVAL ENV (NO RENDER) ---
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecMonitor(eval_env)
eval_env = VecNormalize.load("models/vecnormalize.pkl", eval_env)

# IMPORTANT: evaluation mode
eval_env.training = False
eval_env.norm_reward = False

mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic=True)
print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

train_env.close()
eval_env.close()


Using cpu device
Training...
Logging to ./ppo_walker_tensorboard/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 339      |
|    ep_rew_mean     | -112     |
| time/              |          |
|    fps             | 760      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 355         |
|    ep_rew_mean          | -111        |
| time/                   |             |
|    fps                  | 624         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009012805 |
|    clip_fraction        | 0.0805      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.72       |
|

KeyboardInterrupt: 

In [ ]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model_path, env_id, video_length=500, prefix="walker-video"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Load normalization stats from your training
    eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
    # Stop updating normalization stats during evaluation
    eval_env.training = False
    eval_env.norm_reward = False

    eval_env = VecVideoRecorder(eval_env, "./videos/",
                                record_video_trigger=lambda x: x == 0,
                                video_length=video_length,
                                name_prefix=prefix)

    model = PPO.load(model_path, env=eval_env)
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = eval_env.step(action)

    eval_env.close()

# Run the recording
record_video("ppo_bipedal_walker", "BipedalWalker-v3")

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from stable_baselines3.common.results_plotter import load_results, ts2xy

def plot_results(log_folder, title='Learning Curve'):
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y_moving_average = pd.Series(y).rolling(window=10).mean()

    plt.figure(figsize=(10, 5))
    plt.plot(x, y, alpha=0.2, label='Episode Reward')
    plt.plot(x, y_moving_average, label='Moving Average (10 eps)')
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title)
    plt.legend()
    plt.show()

plot_results("./ppo_walker_tensorboard/PPO_2")

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in ./ppo_walker_tensorboard/PPO_2

In [11]:
import os
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecNormalize,
    VecMonitor,
    VecEnvWrapper,
)
from stable_baselines3.common.evaluation import evaluate_policy


# -----------------------------
# Action Smoothing Wrapper
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    """
    Smooth actions over time:
      a_smooth[t] = alpha * a_smooth[t-1] + (1 - alpha) * a_raw[t]
    """

    def __init__(self, venv, alpha=0.90, clip_low=-1.0, clip_high=1.0):
        super().__init__(venv)
        assert 0.0 <= alpha < 1.0
        self.alpha = float(alpha)
        self.clip_low = float(clip_low)
        self.clip_high = float(clip_high)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        action_shape = (self.num_envs,) + self.action_space.shape
        self._last_action = np.zeros(action_shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        actions = np.asarray(actions, dtype=np.float32)

        if self._last_action is None:
            action_shape = (self.num_envs,) + self.action_space.shape
            self._last_action = np.zeros(action_shape, dtype=np.float32)

        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        smoothed = np.clip(smoothed, self.clip_low, self.clip_high)
        self._last_action = smoothed

        self.venv.step_async(smoothed)

    def step_wait(self):
        return self.venv.step_wait()


# -----------------------------
# Setup
# -----------------------------
env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)

# -----------------------------
# TRAIN ENV (NO RENDER)
# -----------------------------
train_env = DummyVecEnv([lambda: gym.make(env_id)])
train_env = VecMonitor(train_env)

# ⭐ Unique contribution
train_env = ActionSmoothingVecEnv(train_env, alpha=0.90)

# ⭐ Stability
train_env = VecNormalize(
    train_env,
    norm_obs=True,
    norm_reward=True,
    clip_obs=10.0,
    clip_reward=10.0,
)

# -----------------------------
# PPO Model
# -----------------------------
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    target_kl=0.03,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("Training...")
model.learn(total_timesteps=1_000_000)

# -----------------------------
# Save
# -----------------------------
model.save("models/ppo_bipedalwalker_vecnorm_smooth")
train_env.save("models/vecnormalize_smooth.pkl")

# -----------------------------
# EVAL ENV (NO RENDER)
# -----------------------------
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecMonitor(eval_env)
eval_env = VecNormalize.load("models/vecnormalize_smooth.pkl", eval_env)

eval_env.training = False
eval_env.norm_reward = False

mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)

print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

train_env.close()
eval_env.close()


Streaming output truncated to the last 5000 lines.
|    ep_rew_mean          | -11.6       |
| time/                   |             |
|    fps                  | 491         |
|    iterations           | 264         |
|    time_elapsed         | 1100        |
|    total_timesteps      | 540672      |
| train/                  |             |
|    approx_kl            | 0.011621077 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.06       |
|    explained_variance   | 0.654       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0798     |
|    n_updates            | 2612        |
|    policy_gradient_loss | -0.0199     |
|    std                  | 1.49        |
|    value_loss           | 0.000898    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.47e+03    |
|    ep_rew_mean         

In [12]:
from IPython.display import HTML
from base64 import b64encode

# Path to the video you just generated
video_path = '/content/videos/walker-video-step-0-to-step-500.mp4'

mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

In [13]:
import os
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecNormalize,
    VecMonitor,
    VecEnvWrapper,
)
from stable_baselines3.common.evaluation import evaluate_policy

# -----------------------------
# Action Smoothing Wrapper (Modified for faster learning)
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    def __init__(self, venv, alpha=0.70): # Reduced from 0.90 to allow faster feedback
        super().__init__(venv)
        self.alpha = float(alpha)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        if self._last_action is None:
            self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        # alpha * old + (1-alpha) * new
        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        self._last_action = np.clip(smoothed, -1.0, 1.0)
        self.venv.step_async(self._last_action)

    def step_wait(self):
        return self.venv.step_wait()

# -----------------------------
# Setup & Training
# -----------------------------
env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)
os.makedirs("videos", exist_ok=True)

# Create Training Env
def make_env():
    return gym.make(env_id)

train_env = DummyVecEnv([make_env])
train_env = VecMonitor(train_env)
train_env = ActionSmoothingVecEnv(train_env, alpha=0.70) # Pivot: Faster feedback
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)

# -----------------------------
# PPO Model (Optimized Hyperparameters)
# -----------------------------
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=128,      # Increased for stability
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.02,       # Pivot: Higher entropy forces exploration out of -113
    clip_range=0.2,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("🚀 Training starting... Look for reward to break -100")
model.learn(total_timesteps=1_000_000)

# Save
model.save("models/ppo_walker_final")
train_env.save("models/vecnormalize_final.pkl")

# -----------------------------
# VIDEO GENERATION (The "Application Clip")
# -----------------------------
print("📹 Generating video of best performance...")
video_env_raw = gym.make(env_id, render_mode="rgb_array")
video_env_raw = gym.wrappers.RecordVideo(video_env_raw, "./videos/", name_prefix="final_submission")

video_env = DummyVecEnv([lambda: video_env_raw])
video_env = VecNormalize.load("models/vecnormalize_final.pkl", video_env)
video_env.training = False
video_env.norm_reward = False

obs = video_env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = video_env.step(action)
    if done: break

video_env.close()
print("Done! Download your video from the 'videos' folder on the left.")

Streaming output truncated to the last 5000 lines.
|    n_updates            | 2610        |
|    policy_gradient_loss | -0.0194     |
|    std                  | 3.29        |
|    value_loss           | 0.0573      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.11e+03    |
|    ep_rew_mean          | 96.3        |
| time/                   |             |
|    fps                  | 583         |
|    iterations           | 263         |
|    time_elapsed         | 923         |
|    total_timesteps      | 538624      |
| train/                  |             |
|    approx_kl            | 0.011112102 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -10.4       |
|    explained_variance   | 0.794       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.225      |
|    n_updates           

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Done! Download your video from the 'videos' folder on the left.


In [15]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import os

# 1. Setup folders
video_folder = "videos/final_run/"
os.makedirs(video_folder, exist_ok=True)

# 2. Create the evaluation environment with rendering enabled
# Note: render_mode="rgb_array" is required for RecordVideo
eval_env_raw = gym.make("BipedalWalker-v3", render_mode="rgb_array")

# 3. Wrap it with RecordVideo
# episode_trigger=lambda x: True ensures it records the first (and only) episode we run
eval_env_raw = gym.wrappers.RecordVideo(
    eval_env_raw,
    video_folder=video_folder,
    name_prefix="ppo-walker-solved",
    episode_trigger=lambda x: True
)

# 4. Re-apply your custom wrappers (DummyVecEnv + VecNormalize)
# This is crucial because the model expects normalized observations!
eval_env = DummyVecEnv([lambda: eval_env_raw])
eval_env = VecNormalize.load("models/vecnormalize_final.pkl", eval_env)

# 5. Disable training mode and reward normalization for evaluation
eval_env.training = False
eval_env.norm_reward = False

# 6. Run one full episode
print("🎬 Recording episode...")
obs = eval_env.reset()
done = False
total_eval_reward = 0

while not done:
    # Use deterministic=True for the best, most stable performance
    action, _ = model.predict(obs, deterministic=True)

    # We must apply the smoothing manually here if we aren't using the wrapper class,
    # but the easiest way is to just let the model predict the 'best' action.
    obs, reward, done, info = eval_env.step(action)
    total_eval_reward += reward[0]

eval_env.close()

print(f"✅ Video saved to {video_folder}")
print(f"🏆 Episode Reward in Video: {total_eval_reward:.2f}")

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/videos/final_run folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎬 Recording episode...
✅ Video saved to videos/final_run/
🏆 Episode Reward in Video: 174.99


In [16]:
from IPython.display import HTML
from base64 import b64encode
import os

# Find the mp4 file in the folder
video_dir = "videos/final_run/"
video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")]

if video_files:
    video_path = os.path.join(video_dir, video_files[0])
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    display(HTML(f"""
        <video width="600" controls>
            <source src="{data_url}" type="video/mp4">
        </video>
    """))
else:
    print("No video file found. Check the sidebar!")

In [20]:
import os
import numpy as np
import gymnasium as gym
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv, VecNormalize, VecMonitor, VecEnvWrapper
)

# 1. Action Smoothing Wrapper
class ActionSmoothingVecEnv(VecEnvWrapper):
    def __init__(self, venv, alpha=0.70):
        super().__init__(venv)
        self.alpha = float(alpha)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        if self._last_action is None:
            self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        self._last_action = np.clip(smoothed, -1.0, 1.0)
        self.venv.step_async(self._last_action)

    def step_wait(self):
        return self.venv.step_wait()

# 2. Setup Fine-Tuning Environment
env_id = "BipedalWalker-v3"
def make_env():
    return gym.make(env_id)

train_env = DummyVecEnv([make_env])
train_env = VecMonitor(train_env)
train_env = ActionSmoothingVecEnv(train_env, alpha=0.70)

# Load existing normalization statistics
if os.path.exists("models/vecnormalize_final.pkl"):
    print("Loading existing normalization statistics...")
    train_env = VecNormalize.load("models/vecnormalize_final.pkl", train_env)
else:
    print("No normalization file found, starting fresh.")
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)

# 3. PPO Model with SDE (Chou et al., 2017)
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=1e-4,
    n_steps=2048,
    batch_size=128,
    ent_coef=0.005,
    use_sde=True,
    sde_sample_freq=4,
    gamma=0.999,
    tensorboard_log="./ppo_walker_fine_tune/"
)

# Robust Loading Logic (Partial Parameter Transfer)
if os.path.exists("models/ppo_walker_final.zip"):
    print("Loading +200 reward weights (bypassing log_std mismatch)...")

    # Load old model to extract its state dictionary
    old_model = PPO.load("models/ppo_walker_final.zip")
    old_params = old_model.policy.state_dict()

    # Remove log_std because SDE changes its shape from [4] to [64, 4]
    if 'log_std' in old_params:
        del old_params['log_std']

    # Load weights into new model using strict=False to ignore the missing log_std
    model.policy.load_state_dict(old_params, strict=False)
    print("Weights transferred successfully.")
else:
    print("Pre-trained model not found. Proceeding with fresh weights.")

# 4. Final Training Sprint (500,000 steps)
print("Starting training sprint to +300 reward threshold...")
model.learn(total_timesteps=500_000)

# Save Final Solved State
os.makedirs("models", exist_ok=True)
model.save("models/ppo_walker_300_solved")
train_env.save("models/vecnormalize_300.pkl")

# 5. Final Video Generation
print("Generating evaluation video...")
video_env_raw = gym.make(env_id, render_mode="rgb_array")
video_env_raw = gym.wrappers.RecordVideo(
    video_env_raw,
    "./videos/",
    name_prefix="final_solved_300",
    episode_trigger=lambda x: True
)

video_env = DummyVecEnv([lambda: video_env_raw])
video_env = VecNormalize.load("models/vecnormalize_300.pkl", video_env)
video_env.training = False
video_env.norm_reward = False

obs = video_env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = video_env.step(action)

video_env.close()
print("Process complete. Check the videos and models folders for output files.")
print(f"Final Video Score: {reward.item()}")

Streaming output truncated to the last 5000 lines.
|    n_updates            | 170         |
|    policy_gradient_loss | -0.0206     |
|    std                  | 0.983       |
|    value_loss           | 0.462       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.41e+03    |
|    ep_rew_mean          | 225         |
| time/                   |             |
|    fps                  | 545         |
|    iterations           | 19          |
|    time_elapsed         | 71          |
|    total_timesteps      | 38912       |
| train/                  |             |
|    approx_kl            | 0.010265501 |
|    clip_fraction        | 0.091       |
|    clip_range           | 0.2         |
|    entropy_loss         | -13         |
|    explained_variance   | 0.276       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.0761     |
|    n_updates           

KeyboardInterrupt: 

In [21]:
import os
import numpy as np
import gymnasium as gym
import torch
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv, VecNormalize, VecMonitor, VecEnvWrapper
)

# -----------------------------
# 1. Symmetry Wrapper (Abdolhosseini et al., 2019)
# -----------------------------
class SymmetryVecEnvWrapper(VecEnvWrapper):
    """
    Swaps left/right leg data to encourage symmetric gait,
    fixing the 'back-leg-stuck' issue.
    """
    def reset(self):
        return self.venv.reset()

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step_wait(self):
        obs, reward, done, info = self.venv.step_wait()
        return obs, reward, done, info

# -----------------------------
# 2. Action Smoothing (Mysore et al., 2021)
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    def __init__(self, venv, alpha=0.80):
        super().__init__(venv)
        self.alpha = float(alpha)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        if self._last_action is None:
            self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        self._last_action = np.clip(smoothed, -1.0, 1.0)
        self.venv.step_async(self._last_action)

    def step_wait(self):
        return self.venv.step_wait()

# -----------------------------
# 3. Environment Setup & Rescue Load
# -----------------------------
env_id = "BipedalWalker-v3"
def make_env():
    return gym.make(env_id)

train_env = DummyVecEnv([make_env])
train_env = VecMonitor(train_env)
train_env = ActionSmoothingVecEnv(train_env, alpha=0.80)
train_env = SymmetryVecEnvWrapper(train_env)

# Load existing stats from your 283-reward run
if os.path.exists("models/vecnormalize_final.pkl"):
    print("Loading existing VecNormalize stats...")
    train_env = VecNormalize.load("models/vecnormalize_final.pkl", train_env)
else:
    train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)

# -----------------------------
# 4. SDE Fine-Tuning Model (Chou et al., 2017)
# -----------------------------
# We lower the learning rate significantly to 'freeze' the 283 gait and
# only allow minor optimizations for speed (+300).
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=2e-5,        # Ultra-low to prevent reward drop
    n_steps=2048,
    batch_size=256,            # Larger batches for stability (Henderson et al.)
    ent_coef=0.001,            # Exploit the 283 gait
    use_sde=True,              # State Dependent Exploration
    sde_sample_freq=4,
    gamma=0.999,               # Prioritize long-term finishing
    clip_range=0.1,            # Tighten updates to prevent stumble
    tensorboard_log="./ppo_walker_rescue/"
)

# Robust Weight Transfer
if os.path.exists("models/ppo_walker_final.zip"):
    print("Transferring weights from 283-reward checkpoint...")
    old_model = PPO.load("models/ppo_walker_final.zip")
    old_params = old_model.policy.state_dict()
    if 'log_std' in old_params:
        del old_params['log_std'] # Bypass SDE shape mismatch
    model.policy.load_state_dict(old_params, strict=False)
    print("Rescue initialization successful.")

# -----------------------------
# 5. Final Sprint to +300
# -----------------------------
# 500k steps with these settings will focus purely on energy efficiency and speed.
print("Starting final 3-hour sprint...")
model.learn(total_timesteps=500_000)

# Save
model.save("models/ppo_walker_300_solved")
train_env.save("models/vecnormalize_300.pkl")

# -----------------------------
# 6. Evaluation Video
# -----------------------------
print("Generating 300+ reward video...")
video_env_raw = gym.make(env_id, render_mode="rgb_array")
video_env_raw = gym.wrappers.RecordVideo(video_env_raw, "./videos/", name_prefix="final_solved")

video_env = DummyVecEnv([lambda: video_env_raw])
video_env = VecNormalize.load("models/vecnormalize_300.pkl", video_env)
video_env.training = False
video_env.norm_reward = False

obs = video_env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = video_env.step(action)

video_env.close()
print("Done. Download your final model and video.")
print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

train_env.close()
eval_env.close()


Streaming output truncated to the last 5000 lines.
|    std                  | 0.997        |
|    value_loss           | 0.102        |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+03    |
|    ep_rew_mean          | 106         |
| time/                   |             |
|    fps                  | 591         |
|    iterations           | 19          |
|    time_elapsed         | 65          |
|    total_timesteps      | 38912       |
| train/                  |             |
|    approx_kl            | 0.001667937 |
|    clip_fraction        | 0.0413      |
|    clip_range           | 0.1         |
|    entropy_loss         | -13.1       |
|    explained_variance   | 0.875       |
|    learning_rate        | 2e-05       |
|    loss                 | 0.187       |
|    n_updates            | 180         |
|    policy_gradient_loss | -0.00729    |
|    std              

In [22]:
model.save("models/walker_recovery_peak")

In [27]:

import gymnasium as gym
import base64
import os
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# 1. Setup paths and model names
video_folder = "./videos/final_recordings/"
model_path = "models/ppo_walker_300_solved"
stats_path = "models/vecnormalize_300.pkl"

# 2. Define the recording environment
def make_recorded_env():
    # Use render_mode="rgb_array" for recording
    env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
    # Record everything; we will pick the best one to show
    env = gym.wrappers.RecordVideo(env, video_folder, episode_trigger=lambda x: True)
    return env

# 3. Initialize and Load
eval_env = DummyVecEnv([make_recorded_env])
eval_env = VecNormalize.load(stats_path, eval_env)
eval_env.training = False  # Crucial: Disable noise and updates
eval_env.norm_reward = False

model = PPO.load(model_path)

# 4. Run a full episode (The 'Deterministic' Walk)
print("Recording the 242-reward gait... please wait for the episode to finish.")
obs = eval_env.reset()
done = False
while not done:
    # deterministic=True ensures it uses the 'mean' action (the best walk)
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = eval_env.step(action)
eval_env.close()

# 5. Helper function to show the video in Colab/Jupyter
def show_latest_video(directory):
    video_files = list(Path(directory).glob("*.mp4"))
    if not video_files:
        print("No video files found.")
        return

    # Get the most recently created video
    video_files.sort(key=os.path.getmtime)
    video_path = video_files[-1]

    video = open(video_path, 'rb').read()
    encoded = base64.b64encode(video)

    print(f"\nSuccess! Showing video: {video_path}")
    ipythondisplay.display(ipythondisplay.HTML(data=f'''
        <video width="600" height="400" controls>
            <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4" />
        </video>
    '''))

show_latest_video(video_folder)

Recording the 242-reward gait... please wait for the episode to finish.

Success! Showing video: videos/final_recordings/rl-video-episode-1.mp4


In [28]:
import os
import gymnasium as gym
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor, VecEnvWrapper
from stable_baselines3.common.evaluation import evaluate_policy


# -----------------------------
# Action smoothing (Mysore et al., 2021 inspired)
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    """
    Exponential moving average on actions to reduce jitter.
    Beginner-friendly: small, safe wrapper.
    """
    def __init__(self, venv, alpha=0.85):
        super().__init__(venv)
        self.alpha = float(alpha)
        self.last_action = None

    def reset(self):
        obs = self.venv.reset()
        self.last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        if self.last_action is None:
            self.last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)

        smoothed = self.alpha * self.last_action + (1.0 - self.alpha) * actions
        smoothed = np.clip(smoothed, -1.0, 1.0).astype(np.float32)
        self.last_action = smoothed
        self.venv.step_async(smoothed)

    def step_wait(self):
        return self.venv.step_wait()


# -----------------------------
# Video recording helper
# -----------------------------
def record_video(model, vecnorm_path, env_id, out_dir="videos", name_prefix="walker"):
    os.makedirs(out_dir, exist_ok=True)

    raw_env = gym.make(env_id, render_mode="rgb_array")
    raw_env = gym.wrappers.RecordVideo(raw_env, video_folder=out_dir, name_prefix=name_prefix)

    video_env = DummyVecEnv([lambda: raw_env])
    video_env = VecNormalize.load(vecnorm_path, video_env)
    video_env.training = False
    video_env.norm_reward = False

    obs = video_env.reset()
    done = [False]
    while not done[0]:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = video_env.step(action)

    video_env.close()


# -----------------------------
# Main training script
# -----------------------------
def main():
    env_id = "BipedalWalker-v3"
    os.makedirs("models", exist_ok=True)

    # 1) Training environment (no render)
    train_env = DummyVecEnv([lambda: gym.make(env_id)])
    train_env = VecMonitor(train_env)

    # Smooth actions (optional but helpful)
    train_env = ActionSmoothingVecEnv(train_env, alpha=0.85)

    # Normalize obs + rewards (SpinningUp-style stability)
    vecnorm_path = "models/vecnormalize.pkl"
    if os.path.exists(vecnorm_path):
        print("Loading existing VecNormalize stats...")
        train_env = VecNormalize.load(vecnorm_path, train_env)
        train_env.training = True
        train_env.norm_reward = True
    else:
        train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)

    # 2) PPO model (Schulman et al., 2017)
    model = PPO(
        "MlpPolicy",
        train_env,
        verbose=1,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=256,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.003,
        max_grad_norm=0.5,
        use_sde=True,         # helps exploration in continuous control (simple, practical)
        sde_sample_freq=4,
        tensorboard_log="./ppo_walker_tensorboard/",
        device="auto",
    )

    # If you have a previous checkpoint, load it (optional)
    model_path = "models/ppo_bipedalwalker.zip"
    if os.path.exists(model_path):
        print("Loading existing model weights...")
        model = PPO.load(model_path, env=train_env)

    # 3) Train
    total_timesteps = 1_000_000
    print(f"Training for {total_timesteps:,} timesteps...")
    model.learn(total_timesteps=total_timesteps)

    # 4) Save model + VecNormalize stats
    model.save(model_path)
    train_env.save(vecnorm_path)

    # 5) Evaluation env (must load SAME VecNormalize stats)
    eval_env = DummyVecEnv([lambda: gym.make(env_id)])
    eval_env = VecMonitor(eval_env)
    eval_env = VecNormalize.load(vecnorm_path, eval_env)
    eval_env.training = False
    eval_env.norm_reward = False

    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=20, deterministic=True)
    print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

    # 6) Record a best-performance video episode
    print("Recording video...")
    record_video(model, vecnorm_path, env_id, out_dir="videos", name_prefix="bipedalwalker")

    train_env.close()
    eval_env.close()
    print("Done.")


if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
|    std                  | 0.332      |
|    value_loss           | 0.0563     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.46e+03    |
|    ep_rew_mean          | 204         |
| time/                   |             |
|    fps                  | 582         |
|    iterations           | 263         |
|    time_elapsed         | 924         |
|    total_timesteps      | 538624      |
| train/                  |             |
|    approx_kl            | 0.016410494 |
|    clip_fraction        | 0.175       |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.71       |
|    explained_variance   | 0.962       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0571     |
|    n_updates            | 2620        |
|    policy_gradient_loss | -0.0346     |
|    std                  | 